In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
import json
import optuna
from functools import partial
from joblib import Memory

# Setup memory cache
cache_dir = './joblib_cache'
memory = Memory(cache_dir, verbose=0)

class Config:
    TARGET = 'AWS'
    USE_LAG_FEATURES = True
    USE_ROLLING_STATISTICS = True

# Paths
base_path = "/kaggle/input/ai-dataimputedataset-k-fold"
months = ["2019-04", "2019-10", "2020-04", "2020-10"]
folds = [f"fold_{i}" for i in range(1, 6)]

# Define selected features
selected_features = [
    'TCW', 'TCLW', 'R250', 'R500', 'R850', 'U850', 'V850', 'EWSS', 'KX', 'CAPE', 'SSHF', 'PEV'
]

# Device configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model parameters - now configurable via Optuna
MODEL_CONFIG = {
    "04": {"BATCH_SIZE": 64, "EPOCHS": 10},
    "10": {"BATCH_SIZE": 128, "EPOCHS": 15},
}

# Create lag features
@memory.cache
def create_lag_features(train_df, test_df, target_column, lag_steps, groupby_cols):
    """Create lag features for the target column"""
    print("Creating lag features...")
    result_df = test_df.copy()
    
    # Combine train and test for continuous time series
    combined_df = pd.concat([train_df, test_df]).sort_values('DATETIME').reset_index(drop=True)
    
    # Create lag features
    for lag in lag_steps:
        combined_df[f'{target_column}_lag{lag}'] = combined_df.groupby(groupby_cols)[target_column].shift(lag)
    
    # Extract only the test portion with lag features
    result_df = combined_df.iloc[len(train_df):].reset_index(drop=True)
    
    print(f"Created lag features: {[f'{target_column}_lag{lag}' for lag in lag_steps]}")
    return result_df

# Create rolling statistics
@memory.cache
def create_rolling_statistics(train_df, test_df, target_column, window_sizes, groupby_cols):
    """Create rolling statistics features for the target column"""
    print("Creating rolling statistics...")
    result_df = test_df.copy()
    
    # Combine train and test for continuous rolling stats
    combined_df = pd.concat([train_df, test_df]).sort_values('DATETIME').reset_index(drop=True)
    
    # Create rolling features
    for window in window_sizes:
        # Rolling mean
        combined_df[f'{target_column}_rollmean_{window}'] = combined_df.groupby(groupby_cols)[target_column].transform(
            lambda x: x.rolling(window, min_periods=1).mean())
        # Rolling std
        combined_df[f'{target_column}_rollstd_{window}'] = combined_df.groupby(groupby_cols)[target_column].transform(
            lambda x: x.rolling(window, min_periods=1).std())
    
    # Extract only the test portion with rolling features
    result_df = combined_df.iloc[len(train_df):].reset_index(drop=True)
    
    print(f"Created rolling features for windows: {window_sizes}")
    return result_df

# Handle missing values
@memory.cache
def handle_missing_values(df, lag_steps, window_sizes):
    """Handle missing values in the dataframe"""
    print("Handling missing values...")
    result_df = df.copy()
    
    # Fill NaN values in lag features with 0
    for lag in lag_steps:
        lag_col = f'{Config.TARGET}_lag{lag}'
        if lag_col in result_df.columns:
            result_df[lag_col] = result_df[lag_col].fillna(0)
    
    # Fill NaN values in rolling features with 0
    for window in window_sizes:
        mean_col = f'{Config.TARGET}_rollmean_{window}'
        std_col = f'{Config.TARGET}_rollstd_{window}'
        
        if mean_col in result_df.columns:
            result_df[mean_col] = result_df[mean_col].fillna(0)
        
        if std_col in result_df.columns:
            result_df[std_col] = result_df[std_col].fillna(0)
    
    # Fill remaining NaNs with 0
    result_df = result_df.fillna(0)
    
    return result_df



# Modified load_and_process_data function to filter features
@memory.cache
def load_and_process_data(file_path, train_file_path=None, lag_steps=None, window_sizes=None):
    """
    Load and process data from file_path with optional lag features and rolling statistics
    If train_file_path is provided, use it for creating lag and rolling features
    """
    print(f"\U0001F4E5 Loading {file_path}")
    
    try:
        raw_df = pd.read_csv(file_path)
        
        # Filter to keep only selected features plus essential columns
        essential_cols = ['DATETIME', 'ROW', 'COL', Config.TARGET]
        feature_cols = [col for col in selected_features if col in raw_df.columns]
        filtered_cols = essential_cols + feature_cols
        
        # Keep only needed columns
        raw_df = raw_df[filtered_cols]
        
        # Convert target to numeric and handle NaN/inf values
        raw_df[Config.TARGET] = pd.to_numeric(raw_df[Config.TARGET], errors='coerce')
        raw_df = raw_df.replace([np.inf, -np.inf], np.nan)
        df = raw_df.dropna(subset=[Config.TARGET]).copy()
        
        # Sort by datetime for proper sequence handling
        if 'DATETIME' in df.columns:
            df = df.sort_values("DATETIME").reset_index(drop=True)
        
        # If we're in train mode or not creating lag/rolling features
        if train_file_path is None or (lag_steps is None and window_sizes is None):
            return df
        
        # Otherwise, we're in eval mode and need to carefully create features
        train_df = pd.read_csv(train_file_path)
        
        # Filter training data to keep only selected features
        train_df = train_df[filtered_cols]
        
        train_df[Config.TARGET] = pd.to_numeric(train_df[Config.TARGET], errors='coerce')
        train_df = train_df.replace([np.inf, -np.inf], np.nan)
        train_df = train_df.dropna(subset=[Config.TARGET]).copy()
        
        if 'DATETIME' in train_df.columns:
            train_df = train_df.sort_values("DATETIME").reset_index(drop=True)
        
        # Create lag features if needed
        if Config.USE_LAG_FEATURES and lag_steps:
            df = create_lag_features(train_df, df, Config.TARGET, lag_steps, ['ROW', 'COL'])
        
        # Create rolling statistics if needed
        if Config.USE_ROLLING_STATISTICS and window_sizes:
            df = create_rolling_statistics(train_df, df, Config.TARGET, window_sizes, ['ROW', 'COL'])
        
        # Handle missing values
        if lag_steps or window_sizes:
            df = handle_missing_values(df, lag_steps or [], window_sizes or [])
        
        print(f"✅ After processing: {df.shape[0]} rows, {df.shape[1]} columns")
        return df
        
    except Exception as e:
        print(f"❌ Error loading or processing data: {str(e)}")
        return pd.DataFrame()

# # Enhanced LSTM Model
# class LSTMModel(nn.Module):
#     def __init__(self, input_size, hidden_size=64, num_layers=2, dropout=0.0, time_step_out=1):
#         super(LSTMModel, self).__init__()
#         self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
#         self.fc = nn.Linear(hidden_size, time_step_out)
#         self.time_step_out = time_step_out

#     def forward(self, x):
#         out, _ = self.lstm(x)
#         return self.fc(out[:, -1, :])

# Create sequences with configurable input and output time steps
def create_sequences(df, input_cols, target_col, time_step_in, time_step_out=1, stride=1):
    """
    Create sequences from dataframe with configurable input and output time steps
    - time_step_in: number of time steps for input
    - time_step_out: number of future steps to predict
    - stride: step size for sliding window
    """
    sequences, targets = [], []
    grouped = df.groupby(['ROW', 'COL'])
    
    for _, group in grouped:
        # Make sure group is sorted by time
        if 'DATETIME' in group.columns:
            group = group.sort_values("DATETIME")
            
        data = group[input_cols].values
        target_data = group[target_col].values
        
        if len(data) < time_step_in + time_step_out:
            continue
        
        for i in range(0, len(data) - time_step_in - time_step_out + 1, stride):
            seq = data[i:i+time_step_in]
            if time_step_out == 1:
                target = target_data[i+time_step_in]
                targets.append(target)
            else:
                target = target_data[i+time_step_in:i+time_step_in+time_step_out]
                targets.append(target)
            sequences.append(seq)
            
    if not sequences:
        return torch.tensor([]), torch.tensor([])
    
    if time_step_out == 1:
        return torch.tensor(sequences, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32).unsqueeze(1)
    else:
        return torch.tensor(sequences, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32)

# Prepare data loaders
# Modified prepare_data_loaders function
def prepare_data_loaders(month, fold, lag_steps, window_sizes, time_step_in, time_step_out, batch_size, stride=1):
    """Prepare data loaders with specific time steps and features"""
    folder = os.path.join(base_path, month, fold)
    
    # Load train data
    train_df = load_and_process_data(os.path.join(folder, "processed_train.csv"))
    
    # Load validation data
    val_df = load_and_process_data(os.path.join(folder, "processed_val.csv"))
    
    # Load test data
    test_df = load_and_process_data(os.path.join(folder, "merged_test.csv"))
    
    if train_df.empty or val_df.empty or test_df.empty:
        print("⚠️ One or more datasets are empty")
        return None, None, None, 0
    
    # Sort data by time
    if 'DATETIME' in train_df.columns:
        train_df = train_df.sort_values("DATETIME").reset_index(drop=True)
    if 'DATETIME' in val_df.columns:
        val_df = val_df.sort_values("DATETIME").reset_index(drop=True)
    if 'DATETIME' in test_df.columns:
        test_df = test_df.sort_values("DATETIME").reset_index(drop=True)
    
    # Create features before further processing
    # 1. Create lag features separately for each dataset
    if Config.USE_LAG_FEATURES and lag_steps:
        # Create lag features for train set using itself
        print(f"Creating lag features for train set with lags: {lag_steps}")
        for lag in lag_steps:
            train_df[f'{Config.TARGET}_lag{lag}'] = train_df.groupby(['ROW', 'COL'])[Config.TARGET].shift(lag)
        
        # Create lag features for validation set using train + val
        print(f"Creating lag features for val set with lags: {lag_steps}")
        train_val_df = pd.concat([train_df, val_df]).sort_values('DATETIME').reset_index(drop=True)
        for lag in lag_steps:
            train_val_df[f'{Config.TARGET}_lag{lag}'] = train_val_df.groupby(['ROW', 'COL'])[Config.TARGET].shift(lag)
        val_df = train_val_df.iloc[len(train_df):].reset_index(drop=True)
        
        # Create lag features for test set using train + val + test
        print(f"Creating lag features for test set with lags: {lag_steps}")
        full_df = pd.concat([train_df, val_df, test_df]).sort_values('DATETIME').reset_index(drop=True)
        for lag in lag_steps:
            full_df[f'{Config.TARGET}_lag{lag}'] = full_df.groupby(['ROW', 'COL'])[Config.TARGET].shift(lag)
        test_df = full_df.iloc[len(train_df) + len(val_df):].reset_index(drop=True)
    
    # 2. Create rolling statistics separately for each dataset
    if Config.USE_ROLLING_STATISTICS and window_sizes:
        # Create rolling stats for train set
        print(f"Creating rolling stats for train set with windows: {window_sizes}")
        for window in window_sizes:
            train_df[f'{Config.TARGET}_rollmean_{window}'] = train_df.groupby(['ROW', 'COL'])[Config.TARGET].transform(
                lambda x: x.rolling(window, min_periods=1).mean())
            train_df[f'{Config.TARGET}_rollstd_{window}'] = train_df.groupby(['ROW', 'COL'])[Config.TARGET].transform(
                lambda x: x.rolling(window, min_periods=1).std())
        
        # Create rolling stats for validation set
        print(f"Creating rolling stats for val set with windows: {window_sizes}")
        train_val_df = pd.concat([train_df, val_df]).sort_values('DATETIME').reset_index(drop=True)
        for window in window_sizes:
            train_val_df[f'{Config.TARGET}_rollmean_{window}'] = train_val_df.groupby(['ROW', 'COL'])[Config.TARGET].transform(
                lambda x: x.rolling(window, min_periods=1).mean())
            train_val_df[f'{Config.TARGET}_rollstd_{window}'] = train_val_df.groupby(['ROW', 'COL'])[Config.TARGET].transform(
                lambda x: x.rolling(window, min_periods=1).std())
        val_df = train_val_df.iloc[len(train_df):].reset_index(drop=True)
        
        # Create rolling stats for test set
        print(f"Creating rolling stats for test set with windows: {window_sizes}")
        full_df = pd.concat([train_df, val_df, test_df]).sort_values('DATETIME').reset_index(drop=True)
        for window in window_sizes:
            full_df[f'{Config.TARGET}_rollmean_{window}'] = full_df.groupby(['ROW', 'COL'])[Config.TARGET].transform(
                lambda x: x.rolling(window, min_periods=1).mean())
            full_df[f'{Config.TARGET}_rollstd_{window}'] = full_df.groupby(['ROW', 'COL'])[Config.TARGET].transform(
                lambda x: x.rolling(window, min_periods=1).std())
        test_df = full_df.iloc[len(train_df) + len(val_df):].reset_index(drop=True)
    
    # Handle missing values
    train_df = train_df.fillna(0)
    val_df = val_df.fillna(0)
    test_df = test_df.fillna(0)
    
    # Prepare feature columns - exclude DATETIME, ROW, COL and target column
    # Include selected_features that are in the DataFrame
    basic_cols = [col for col in selected_features if col in train_df.columns]
    lag_cols = [f'{Config.TARGET}_lag{lag}' for lag in lag_steps if f'{Config.TARGET}_lag{lag}' in train_df.columns]
    roll_cols = []
    
    for window in window_sizes:
        mean_col = f'{Config.TARGET}_rollmean_{window}'
        std_col = f'{Config.TARGET}_rollstd_{window}'
        if mean_col in train_df.columns:
            roll_cols.append(mean_col)
        if std_col in train_df.columns:
            roll_cols.append(std_col)
    
    feature_cols = basic_cols + lag_cols + roll_cols
    
    # Check if we have any features
    if not feature_cols:
        print("⚠️ No features detected! Creating default lag feature.")
        # Create at least one default lag feature
        default_lag = 1
        for df in [train_df, val_df, test_df]:
            df[f'{Config.TARGET}_lag{default_lag}'] = df.groupby(['ROW', 'COL'])[Config.TARGET].shift(default_lag)
            df = df.fillna(0)
        
        feature_cols = [f'{Config.TARGET}_lag{default_lag}']
    
    print(f"Using {len(feature_cols)} features: {feature_cols}")
    
    # Create sequences
    train_x, train_y = create_sequences(train_df, feature_cols, Config.TARGET, time_step_in, time_step_out, stride)
    val_x, val_y = create_sequences(val_df, feature_cols, Config.TARGET, time_step_in, time_step_out, stride)
    test_x, test_y = create_sequences(test_df, feature_cols, Config.TARGET, time_step_in, time_step_out, stride)
    
    if train_x.numel() == 0 or val_x.numel() == 0 or test_x.numel() == 0:
        print("⚠️ One or more sequence sets are empty")
        return None, None, None, 0
    
    print(f"✅ Data shapes - Train: {train_x.shape}, Val: {val_x.shape}, Test: {test_x.shape}")
    
    # Create data loaders
    train_loader = DataLoader(TensorDataset(train_x, train_y), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(val_x, val_y), batch_size=batch_size)
    test_loader = DataLoader(TensorDataset(test_x, test_y), batch_size=batch_size)
    
    return train_loader, val_loader, test_loader, train_x.shape[2]

In [2]:
# Hybrid LSTM Classification-Regression Model
import torch
import torch.nn as nn
import torch.nn.functional as F

class HybridLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, dropout=0.0, time_step_out=1):
        super(HybridLSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        
        # Shared feature extraction
        self.feature_layer = nn.Linear(hidden_size, hidden_size // 2)
        
        # Binary classification: rain or no rain
        self.rain_classifier = nn.Linear(hidden_size // 2, 1)  # Binary output (0=no rain, 1=rain)
        
        # Rain intensity classifier (conditionally used)
        self.intensity_classifier = nn.Linear(hidden_size // 2, 3)  # 3 classes: small, medium, heavy
        
        # Regression head for rain volume (conditionally used)
        self.regressor = nn.Linear(hidden_size // 2, time_step_out)
        
        self.time_step_out = time_step_out
        self.hidden_size = hidden_size

    def forward(self, x):
        # LSTM feature extraction
        lstm_out, _ = self.lstm(x)
        last_out = lstm_out[:, -1, :]  # Get the last time step output
        
        # Extract shared features
        features = F.relu(self.feature_layer(last_out))
        
        # Binary classification: rain or no rain
        rain_logits = self.rain_classifier(features)
        rain_prob = torch.sigmoid(rain_logits)  # Probability of rain
        
        # Rain intensity classification (3 classes)
        intensity_logits = self.intensity_classifier(features)
        intensity_probs = F.softmax(intensity_logits, dim=1)  # Probabilities for each intensity class
        
        # Regression for rain amount
        regression_out = self.regressor(features)
        
        return {
            'rain_logits': rain_logits,
            'rain_prob': rain_prob,
            'intensity_logits': intensity_logits,
            'intensity_probs': intensity_probs,
            'regression': regression_out
        }

# Custom loss function that combines binary classification, multiclass classification, and regression
class HybridRainfallLoss(nn.Module):
    def __init__(self, rain_threshold=0.1, class_boundaries=[0.0, 2.5, 6.0], 
                 binary_weight=1.0, intensity_weight=1.0, regression_weight=1.0):
        super(HybridRainfallLoss, self).__init__()
        self.rain_threshold = rain_threshold
        self.class_boundaries = class_boundaries
        
        # Loss weights
        self.binary_weight = binary_weight
        self.intensity_weight = intensity_weight
        self.regression_weight = regression_weight
        
        # Individual loss functions
        self.binary_loss_fn = nn.BCEWithLogitsLoss(reduction='mean')
        self.intensity_loss_fn = nn.CrossEntropyLoss(reduction='mean')
        self.regression_loss_fn = nn.MSELoss(reduction='mean')
    
    def forward(self, outputs, targets):
        # Extract model outputs
        rain_logits = outputs['rain_logits']
        intensity_logits = outputs['intensity_logits']
        regression = outputs['regression']
        
        # Prepare target values
        # Binary rain targets (1 if rain > threshold, 0 otherwise)
        binary_targets = (targets > self.rain_threshold).float()
        
        # Intensity class targets (0=small, 1=medium, 2=heavy)
        # Create class masks
        small_mask = (targets > self.class_boundaries[0]) & (targets <= self.class_boundaries[1])
        medium_mask = (targets > self.class_boundaries[1]) & (targets <= self.class_boundaries[2])
        heavy_mask = (targets > self.class_boundaries[2])
        
        # Convert to class indices (0, 1, or 2)
        intensity_targets = torch.zeros_like(targets, dtype=torch.long)
        intensity_targets[medium_mask] = 1
        intensity_targets[heavy_mask] = 2
        
        # Only consider intensity loss for samples where rain exists
        valid_intensity_mask = binary_targets.bool().squeeze()
        
        # Calculate losses
        binary_loss = self.binary_loss_fn(rain_logits, binary_targets)
        
        # Only calculate intensity loss on rainy samples (if any exist)
        if valid_intensity_mask.sum() > 0:
            intensity_loss = self.intensity_loss_fn(
                intensity_logits[valid_intensity_mask], 
                intensity_targets[valid_intensity_mask].squeeze()
            )
        else:
            intensity_loss = torch.tensor(0.0, device=rain_logits.device)
        
        # For regression, we can either:
        # 1. Apply regression loss to all samples
        # 2. Apply regression loss only to rainy samples
        # Let's use option 2 for better focus on rain prediction
        if valid_intensity_mask.sum() > 0:
            regression_loss = self.regression_loss_fn(
                regression[valid_intensity_mask], 
                targets[valid_intensity_mask]
            )
        else:
            regression_loss = torch.tensor(0.0, device=rain_logits.device)
        
        # Combine losses with weights
        total_loss = (
            self.binary_weight * binary_loss +
            self.intensity_weight * intensity_loss +
            self.regression_weight * regression_loss
        )
        
        return {
            'total_loss': total_loss,
            'binary_loss': binary_loss.item(),
            'intensity_loss': intensity_loss.item() if isinstance(intensity_loss, torch.Tensor) else intensity_loss,
            'regression_loss': regression_loss.item() if isinstance(regression_loss, torch.Tensor) else regression_loss
        }

# Function to get final predictions combining classification and regression
def get_hybrid_predictions(outputs):
    rain_prob = outputs['rain_prob']
    intensity_probs = outputs['intensity_probs']
    regression = outputs['regression']
    
    # Binary decision: rain or no rain
    is_rain = rain_prob > 0.5
    
    # Get the most likely intensity class
    _, intensity_class = torch.max(intensity_probs, dim=1)
    
    # Initialize predictions with zeros (no rain)
    final_predictions = torch.zeros_like(regression)
    
    # Where rain is predicted, use regression values
    final_predictions[is_rain] = regression[is_rain]
    
    return {
        'is_rain': is_rain,
        'intensity_class': intensity_class,
        'regression': regression,
        'final_predictions': final_predictions
    }

In [3]:
# Modified train_model function for hybrid model
def train_hybrid_model(model, loader, optimizer, loss_fn, epochs, patience=3):
    """Train hybrid model with early stopping"""
    model.train()
    best_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(epochs):
        running_loss = 0.0
        running_binary_loss = 0.0
        running_intensity_loss = 0.0
        running_regression_loss = 0.0
        
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            optimizer.zero_grad()
            
            # Get model outputs
            outputs = model(xb)
            
            # Calculate loss
            loss_dict = loss_fn(outputs, yb)
            total_loss = loss_dict['total_loss']
            
            # Backpropagation
            total_loss.backward()
            optimizer.step()
            
            # Track losses
            running_loss += total_loss.item() * xb.size(0)
            running_binary_loss += loss_dict['binary_loss'] * xb.size(0)
            running_intensity_loss += loss_dict['intensity_loss'] * xb.size(0)
            running_regression_loss += loss_dict['regression_loss'] * xb.size(0)
        
        # Calculate average losses
        avg_loss = running_loss / len(loader.dataset)
        avg_binary_loss = running_binary_loss / len(loader.dataset)
        avg_intensity_loss = running_intensity_loss / len(loader.dataset)
        avg_regression_loss = running_regression_loss / len(loader.dataset)
        
        print(f"  🌀 Epoch {epoch+1}/{epochs}, Total Loss: {avg_loss:.4f}, "
              f"Binary: {avg_binary_loss:.4f}, Intensity: {avg_intensity_loss:.4f}, "
              f"Regression: {avg_regression_loss:.4f}")
        
        # Early stopping check
        if avg_loss < best_loss:
            best_loss = avg_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"  🛑 Early stopping at epoch {epoch+1}")
                break
    
    return best_loss

# Modified evaluate_model function for hybrid model
def evaluate_hybrid_model(model, loader, rain_threshold=0.1):
    """Evaluate hybrid model and return metrics"""
    model.eval()
    all_preds = []
    all_targets = []
    all_binary_preds = []
    all_binary_targets = []
    all_intensity_preds = []
    all_intensity_targets = []
    
    # Classification thresholds for defining rain intensity classes
    class_boundaries = [0.0, 2.5, 6.0]
    
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(DEVICE)
            outputs = model(xb)
            
            # Get predictions
            pred_dict = get_hybrid_predictions(outputs)
            final_preds = pred_dict['final_predictions'].cpu()
            is_rain = pred_dict['is_rain'].cpu()
            intensity_class = pred_dict['intensity_class'].cpu()
            
            # Get binary targets (rain/no rain)
            binary_targets = (yb > rain_threshold).float()
            
            # Get intensity class targets
            intensity_targets = torch.zeros_like(yb, dtype=torch.long)
            intensity_targets[(yb > class_boundaries[0]) & (yb <= class_boundaries[1])] = 0  # Small rain
            intensity_targets[(yb > class_boundaries[1]) & (yb <= class_boundaries[2])] = 1  # Medium rain
            intensity_targets[yb > class_boundaries[2]] = 2  # Heavy rain
            
            # Store predictions and targets
            all_preds.append(final_preds)
            all_targets.append(yb)
            all_binary_preds.append(is_rain)
            all_binary_targets.append(binary_targets)
            all_intensity_preds.append(intensity_class)
            all_intensity_targets.append(intensity_targets)
    
    # Concatenate results
    all_preds = torch.cat(all_preds).squeeze().numpy()
    all_targets = torch.cat(all_targets).squeeze().numpy()
    all_binary_preds = torch.cat(all_binary_preds).squeeze().numpy()
    all_binary_targets = torch.cat(all_binary_targets).squeeze().numpy()
    all_intensity_preds = torch.cat(all_intensity_preds).squeeze().numpy()
    all_intensity_targets = torch.cat(all_intensity_targets).squeeze().numpy()
    
    # Calculate regression metrics
    rmse = mean_squared_error(all_targets, all_preds, squared=False)
    bias = np.mean(all_preds - all_targets)
    r2 = r2_score(all_targets, all_preds)
    
    # Calculate classification metrics
    binary_accuracy = np.mean(all_binary_preds == all_binary_targets)
    
    # Calculate Critical Success Index (CSI) for rain detection
    tp = np.sum((all_binary_preds == 1) & (all_binary_targets == 1))
    fp = np.sum((all_binary_preds == 1) & (all_binary_targets == 0))
    fn = np.sum((all_binary_preds == 0) & (all_binary_targets == 1))
    csi = tp / (tp + fp + fn + 1e-9)
    
    # Calculate intensity classification accuracy (only for actual rain samples)
    rain_mask = all_binary_targets == 1
    if np.sum(rain_mask) > 0:
        intensity_accuracy = np.mean(all_intensity_preds[rain_mask] == all_intensity_targets[rain_mask])
    else:
        intensity_accuracy = np.nan
    
    # Return all metrics
    metrics = {
        'rmse': rmse,
        'bias': bias,
        'r2': r2,
        'csi': csi,
        'binary_accuracy': binary_accuracy,
        'intensity_accuracy': intensity_accuracy
    }
    
    return metrics

In [4]:
# Modified objective function for Optuna optimization with hybrid model
def hybrid_objective(trial, month):
    """Optuna objective function for hybrid model using cross-validation across folds"""
    # Hyperparameters to tune
    hidden_size = trial.suggest_int("hidden_size", 32, 256)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    dropout = trial.suggest_float("dropout", 0.0, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    time_step_in = trial.suggest_int("time_step_in", 12, 48)
    time_step_out = trial.suggest_int("time_step_out", 1, 6)
    stride = trial.suggest_int("stride", 1, 6)
    
    # Loss weights
    binary_weight = trial.suggest_float("binary_weight", 0.5, 2.0)
    intensity_weight = trial.suggest_float("intensity_weight", 0.5, 2.0) 
    regression_weight = trial.suggest_float("regression_weight", 0.5, 2.0)
    
    # Rain threshold (for binary classification)
    rain_threshold = trial.suggest_float("rain_threshold", 0.05, 0.2)
    
    # Lag steps and window sizes
    num_lags = trial.suggest_int("num_lags", 1, 3)
    lag_steps = [trial.suggest_int(f"lag_{i}", 1, 12) for i in range(num_lags)]
    
    num_windows = trial.suggest_int("num_windows", 1, 3)
    window_sizes = [trial.suggest_int(f"window_{i}", 3, 24) for i in range(num_windows)]
    
    # Get batch size from config
    config_key = month.split("-")[1]
    config = MODEL_CONFIG[config_key]
    batch_size = config["BATCH_SIZE"]
    
    # Cross-validation across folds
    fold_val_rmses = []
    
    for fold in folds:
        try:
            # Prepare data loaders
            train_loader, val_loader, _, input_size = prepare_data_loaders(
                month, fold, lag_steps, window_sizes, time_step_in, time_step_out, batch_size, stride
            )
            
            if train_loader is None or val_loader is None:
                # Skip this fold if data preparation failed
                continue
            
            # Create hybrid model with trial parameters
            model = HybridLSTMModel(
                input_size=input_size, 
                hidden_size=hidden_size, 
                num_layers=num_layers, 
                dropout=dropout,
                time_step_out=time_step_out
            ).to(DEVICE)
            
            # Class boundaries for rainfall intensity
            class_boundaries = [0.0, 2.5, 6.0]
            
            # Create hybrid loss function
            loss_fn = HybridRainfallLoss(
                rain_threshold=rain_threshold,
                class_boundaries=class_boundaries,
                binary_weight=binary_weight,
                intensity_weight=intensity_weight,
                regression_weight=regression_weight
            )
            
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)
            
            # Train for a small number of epochs during hyperparameter search
            train_hybrid_model(model, train_loader, optimizer, loss_fn, epochs=5, patience=2)
            
            # Evaluate on validation set
            val_metrics = evaluate_hybrid_model(model, val_loader, rain_threshold=rain_threshold)
            val_rmse = val_metrics['rmse']
            fold_val_rmses.append(val_rmse)
            
        except Exception as e:
            print(f"Error in fold {fold}: {str(e)}")
            continue
    
    # Return mean validation RMSE across folds
    if fold_val_rmses:
        mean_val_rmse = sum(fold_val_rmses) / len(fold_val_rmses)
        return mean_val_rmse
    else:
        return float('inf')

# Function for final evaluation on test set with hybrid model
def evaluate_hybrid_on_test(month, fold, best_params):
    """Train and evaluate hybrid model with best parameters on test set"""
    print(f"\n🚀 Training hybrid model for {month}, {fold} with best parameters")
    
    # Extract parameters
    hidden_size = best_params["hidden_size"]
    num_layers = best_params["num_layers"]
    dropout = best_params["dropout"]
    lr = best_params["lr"]
    time_step_in = best_params["time_step_in"]
    time_step_out = best_params["time_step_out"]
    stride = best_params["stride"]
    
    # Loss weights
    binary_weight = best_params["binary_weight"]
    intensity_weight = best_params["intensity_weight"] 
    regression_weight = best_params["regression_weight"]
    
    # Rain threshold
    rain_threshold = best_params["rain_threshold"]
    
    # Class boundaries for rainfall intensity
    class_boundaries = [0.0, 2.5, 6.0]
    
    # Extract lag steps and window sizes
    num_lags = best_params["num_lags"]
    lag_steps = [best_params[f"lag_{i}"] for i in range(num_lags)]
    
    num_windows = best_params["num_windows"]
    window_sizes = [best_params[f"window_{i}"] for i in range(num_windows)]
    
    # Get config
    config_key = month.split("-")[1]
    config = MODEL_CONFIG[config_key]
    batch_size = config["BATCH_SIZE"]
    epochs = config["EPOCHS"]
    
    # Prepare data
    train_loader, val_loader, test_loader, input_size = prepare_data_loaders(
        month, fold, lag_steps, window_sizes, time_step_in, time_step_out, batch_size, stride
    )
    
    if train_loader is None or val_loader is None or test_loader is None:
        print("❌ Failed to prepare data")
        return None
    
    # Create hybrid model
    model = HybridLSTMModel(
        input_size=input_size,
        hidden_size=hidden_size,
        num_layers=num_layers,
        dropout=dropout,
        time_step_out=time_step_out
    ).to(DEVICE)
    
    # Create hybrid loss function
    loss_fn = HybridRainfallLoss(
        rain_threshold=rain_threshold,
        class_boundaries=class_boundaries,
        binary_weight=binary_weight,
        intensity_weight=intensity_weight,
        regression_weight=regression_weight
    )
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    # Train model on train set
    print(f"🏋️ Training hybrid model on train set...")
    train_hybrid_model(model, train_loader, optimizer, loss_fn, epochs=epochs, patience=5)
    
    # Evaluate on validation set
    print("📊 Evaluating on validation set...")
    val_metrics = evaluate_hybrid_model(model, val_loader, rain_threshold=rain_threshold)
    print(f"  Val RMSE: {val_metrics['rmse']:.4f}, Bias: {val_metrics['bias']:.4f}, "
          f"R²: {val_metrics['r2']:.4f}, CSI: {val_metrics['csi']:.4f}")
    print(f"  Binary Accuracy: {val_metrics['binary_accuracy']:.4f}, "
          f"Intensity Accuracy: {val_metrics['intensity_accuracy']:.4f}")
    
    # Evaluate on test set
    print("🧪 Evaluating on test set...")
    test_metrics = evaluate_hybrid_model(model, test_loader, rain_threshold=rain_threshold)
    print(f"  Test RMSE: {test_metrics['rmse']:.4f}, Bias: {test_metrics['bias']:.4f}, "
          f"R²: {test_metrics['r2']:.4f}, CSI: {test_metrics['csi']:.4f}")
    print(f"  Binary Accuracy: {test_metrics['binary_accuracy']:.4f}, "
          f"Intensity Accuracy: {test_metrics['intensity_accuracy']:.4f}")
    
    # Save model for this fold
    model_path = f"hybrid_model_{month}_{fold}.pt"
    torch.save(model.state_dict(), model_path)
    print(f"💾 Model saved to {model_path}")
    
    # Return results
    return {
        "month": month,
        "fold": fold,
        "test_rmse": round(test_metrics['rmse'], 4),
        "test_bias": round(test_metrics['bias'], 4),
        "test_r2": round(test_metrics['r2'], 4),
        "test_csi": round(test_metrics['csi'], 4),
        "test_binary_acc": round(test_metrics['binary_accuracy'], 4),
        "test_intensity_acc": round(test_metrics['intensity_accuracy'], 4) if not np.isnan(test_metrics['intensity_accuracy']) else None,
        "val_rmse": round(val_metrics['rmse'], 4),
        "val_bias": round(val_metrics['bias'], 4),
        "val_r2": round(val_metrics['r2'], 4),
        "val_csi": round(val_metrics['csi'], 4),
        "val_binary_acc": round(val_metrics['binary_accuracy'], 4),
        "val_intensity_acc": round(val_metrics['intensity_accuracy'], 4) if not np.isnan(val_metrics['intensity_accuracy']) else None,
        "model": model
    }

# Updated main execution function for hybrid model
def run_hybrid_optuna_optimization():
    """Run Optuna optimization for hybrid model with cross-validation and final test evaluation"""
    month_results = {}
    
    for month in months:
        print(f"\n📅 Processing month: {month}")
        
        # Step 1: Find the best hyperparameters using cross-validation
        study_name = f"hybrid_{month}_study"
        study = optuna.create_study(direction="minimize", study_name=study_name)
        
        # Create a partial function with fixed month
        objective_func = partial(hybrid_objective, month=month)
        
        # Run optimization with cross-validation
        try:
            print(f"🔍 Finding optimal hyperparameters for hybrid model on {month} using cross-validation...")
            study.optimize(objective_func, n_trials=20, timeout=3600)  # 1 hour timeout
            
            best_params = study.best_params
            print(f"✅ Best parameters for {month}:")
            for key, value in best_params.items():
                print(f"  {key}: {value}")
            
            # Save best parameters for the month
            best_params_path = f"hybrid_best_params_{month}.json"
            with open(best_params_path, "w") as f:
                json.dump(best_params, f, indent=2)
            
            print(f"💾 Best parameters saved to {best_params_path}")
            
            # Step 2: Final evaluation on test set for each fold
            fold_test_results = []
            for fold in folds:
                print(f"\n📦 Final evaluation on {month}, {fold} test set")
                result = evaluate_hybrid_on_test(month, fold, best_params)
                if result:
                    fold_test_results.append(result)
            
            # Calculate mean performance across folds
            if fold_test_results:
                mean_rmse = sum(r["test_rmse"] for r in fold_test_results) / len(fold_test_results)
                mean_bias = sum(r["test_bias"] for r in fold_test_results) / len(fold_test_results)
                mean_r2 = sum(r["test_r2"] for r in fold_test_results) / len(fold_test_results)
                mean_csi = sum(r["test_csi"] for r in fold_test_results) / len(fold_test_results)
                
                # Calculate mean classification accuracies
                mean_binary_acc = sum(r["test_binary_acc"] for r in fold_test_results) / len(fold_test_results)
                
                # Handle potential None values for intensity accuracy
                intensity_accuracies = [r["test_intensity_acc"] for r in fold_test_results if r["test_intensity_acc"] is not None]
                mean_intensity_acc = sum(intensity_accuracies) / len(intensity_accuracies) if intensity_accuracies else None
                
                # Save best fold result (lowest RMSE)
                best_fold_result = min(fold_test_results, key=lambda x: x["test_rmse"])
                best_model = best_fold_result["model"]
                best_model_path = f"hybrid_best_model_{month}.pt"
                torch.save(best_model.state_dict(), best_model_path)
                
                # Store results for this month
                month_results[month] = {
                    "month": month,
                    "best_fold": best_fold_result["fold"],
                    "best_test_rmse": best_fold_result["test_rmse"],
                    "best_test_bias": best_fold_result["test_bias"],
                    "best_test_r2": best_fold_result["test_r2"],
                    "best_test_csi": best_fold_result["test_csi"],
                    "best_test_binary_acc": best_fold_result["test_binary_acc"],
                    "best_test_intensity_acc": best_fold_result["test_intensity_acc"],
                    "mean_test_rmse": round(mean_rmse, 4),
                    "mean_test_bias": round(mean_bias, 4),
                    "mean_test_r2": round(mean_r2, 4),
                    "mean_test_csi": round(mean_csi, 4),
                    "mean_test_binary_acc": round(mean_binary_acc, 4),
                    "mean_test_intensity_acc": round(mean_intensity_acc, 4) if mean_intensity_acc is not None else None,
                    "best_params": best_params,
                    "fold_results": fold_test_results
                }
                
                print(f"\n📊 Mean test performance for {month}:")
                print(f"  RMSE: {mean_rmse:.4f}, Bias: {mean_bias:.4f}, R²: {mean_r2:.4f}, CSI: {mean_csi:.4f}")
                print(f"  Binary Acc: {mean_binary_acc:.4f}, Intensity Acc: {mean_intensity_acc:.4f if mean_intensity_acc is not None else 'N/A'}")
                print(f"📊 Best fold test performance for {month} (fold {best_fold_result['fold']}):")
                print(f"  RMSE: {best_fold_result['test_rmse']:.4f}, Bias: {best_fold_result['test_bias']:.4f}, "
                      f"R²: {best_fold_result['test_r2']:.4f}, CSI: {best_fold_result['test_csi']:.4f}")
                print(f"💾 Best hybrid model for {month} saved to {best_model_path}")
            
        except Exception as e:
            print(f"❌ Error during optimization for {month}: {str(e)}")
            continue
    
    # Calculate overall mean performance across months
    if month_results:
        overall_mean_rmse = sum(r["best_test_rmse"] for r in month_results.values()) / len(month_results)
        overall_mean_bias = sum(r["best_test_bias"] for r in month_results.values()) / len(month_results)
        overall_mean_r2 = sum(r["best_test_r2"] for r in month_results.values()) / len(month_results)
        overall_mean_csi = sum(r["best_test_csi"] for r in month_results.values()) / len(month_results)
        overall_mean_binary_acc = sum(r["best_test_binary_acc"] for r in month_results.values()) / len(month_results)
        
        # Handle potential None values for intensity accuracy
        intensity_accuracies = [r["best_test_intensity_acc"] for r in month_results.values() if r["best_test_intensity_acc"] is not None]
        overall_mean_intensity_acc = sum(intensity_accuracies) / len(intensity_accuracies) if intensity_accuracies else None
        
        print("\n🏆 Overall mean performance of hybrid model across all months:")
        print(f"  RMSE: {overall_mean_rmse:.4f}, Bias: {overall_mean_bias:.4f}, R²: {overall_mean_r2:.4f}, CSI: {overall_mean_csi:.4f}")
        intensity_acc_str = format(overall_mean_intensity_acc, '.4f') if overall_mean_intensity_acc is not None else 'N/A'
        print(f"  Binary Accuracy: {overall_mean_binary_acc:.4f}, Intensity Accuracy: {intensity_acc_str}")
    
    # Create a dataframe with month results
    month_results_df = pd.DataFrame([
        {
            "month": month,
            "best_fold": data["best_fold"],
            "best_test_rmse": data["best_test_rmse"],
            "best_test_bias": data["best_test_bias"],
            "best_test_r2": data["best_test_r2"],
            "best_test_csi": data["best_test_csi"],
            "best_test_binary_acc": data["best_test_binary_acc"],
            "best_test_intensity_acc": data["best_test_intensity_acc"],
            "mean_test_rmse": data["mean_test_rmse"],
            "mean_test_bias": data["mean_test_bias"],
            "mean_test_r2": data["mean_test_r2"],
            "mean_test_csi": data["mean_test_csi"],
            "mean_test_binary_acc": data["mean_test_binary_acc"],
            "mean_test_intensity_acc": data["mean_test_intensity_acc"],
        } for month, data in month_results.items()
    ])
    
    # Display month-wise results
    print("\n📊 Results by Month:")
    print(month_results_df)
    
    # Save results
    month_results_df.to_csv("hybrid_month_results.csv", index=False)
    
    # Save detailed results
    all_fold_results = []
    for month, data in month_results.items():
        for fold_result in data["fold_results"]:
            all_fold_results.append({
                "month": month,
                "fold": fold_result["fold"],
                "test_rmse": fold_result["test_rmse"],
                "test_bias": fold_result["test_bias"],
                "test_r2": fold_result["test_r2"],
                "test_csi": fold_result["test_csi"],
                "test_binary_acc": fold_result["test_binary_acc"],
                "test_intensity_acc": fold_result["test_intensity_acc"],
                "val_rmse": fold_result["val_rmse"],
                "val_bias": fold_result["val_bias"],
                "val_r2": fold_result["val_r2"],
                "val_csi": fold_result["val_csi"],
                "val_binary_acc": fold_result["val_binary_acc"],
                "val_intensity_acc": fold_result["val_intensity_acc"] if "val_intensity_acc" in fold_result else None,
            })
    
    all_fold_results_df = pd.DataFrame(all_fold_results)
    all_fold_results_df.to_csv("hybrid_all_fold_results.csv", index=False)
    
    print("💾 Results saved to hybrid_month_results.csv and hybrid_all_fold_results.csv")
    
    return month_results

In [5]:
# Execute optimization
if __name__ == "__main__":
    print(f"Running on device: {DEVICE}")
    run_hybrid_optuna_optimization()

[I 2025-04-23 16:29:07,151] A new study created in memory with name: hybrid_2019-04_study


Running on device: cuda

📅 Processing month: 2019-04
🔍 Finding optimal hyperparameters for hybrid model on 2019-04 using cross-validation...
📥 Loading /kaggle/input/ai-dataimputedataset-k-fold/2019-04/fold_1/processed_train.csv
📥 Loading /kaggle/input/ai-dataimputedataset-k-fold/2019-04/fold_1/processed_val.csv
📥 Loading /kaggle/input/ai-dataimputedataset-k-fold/2019-04/fold_1/merged_test.csv
Creating lag features for train set with lags: [5, 8]
Creating lag features for val set with lags: [5, 8]
Creating lag features for test set with lags: [5, 8]
Creating rolling stats for train set with windows: [22, 3, 7]
Creating rolling stats for val set with windows: [22, 3, 7]
Creating rolling stats for test set with windows: [22, 3, 7]
Using 20 features: ['TCW', 'TCLW', 'R250', 'R500', 'R850', 'U850', 'V850', 'EWSS', 'KX', 'CAPE', 'SSHF', 'PEV', 'AWS_lag5', 'AWS_lag8', 'AWS_rollmean_22', 'AWS_rollstd_22', 'AWS_rollmean_3', 'AWS_rollstd_3', 'AWS_rollmean_7', 'AWS_rollstd_7']


<ipython-input-1-eb77431f8768>:232: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(sequences, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32)


✅ Data shapes - Train: torch.Size([2672, 47, 20]), Val: torch.Size([2672, 47, 20]), Test: torch.Size([3518, 47, 20])
Error in fold fold_1: Target size (torch.Size([64, 6])) must be the same as input size (torch.Size([64, 1]))
📥 Loading /kaggle/input/ai-dataimputedataset-k-fold/2019-04/fold_2/processed_train.csv
📥 Loading /kaggle/input/ai-dataimputedataset-k-fold/2019-04/fold_2/processed_val.csv
📥 Loading /kaggle/input/ai-dataimputedataset-k-fold/2019-04/fold_2/merged_test.csv
Creating lag features for train set with lags: [5, 8]
Creating lag features for val set with lags: [5, 8]
Creating lag features for test set with lags: [5, 8]
Creating rolling stats for train set with windows: [22, 3, 7]
Creating rolling stats for val set with windows: [22, 3, 7]
Creating rolling stats for test set with windows: [22, 3, 7]
Using 20 features: ['TCW', 'TCLW', 'R250', 'R500', 'R850', 'U850', 'V850', 'EWSS', 'KX', 'CAPE', 'SSHF', 'PEV', 'AWS_lag5', 'AWS_lag8', 'AWS_rollmean_22', 'AWS_rollstd_22', 'AWS

[I 2025-04-23 16:29:52,550] Trial 0 finished with value: inf and parameters: {'hidden_size': 218, 'num_layers': 2, 'dropout': 0.2357369830035339, 'lr': 0.0002262049708241428, 'time_step_in': 47, 'time_step_out': 6, 'stride': 6, 'binary_weight': 0.6485480579528948, 'intensity_weight': 1.3147993682049501, 'regression_weight': 0.8944303970857828, 'rain_threshold': 0.17846345895484356, 'num_lags': 2, 'lag_0': 5, 'lag_1': 8, 'num_windows': 3, 'window_0': 22, 'window_1': 3, 'window_2': 7}. Best is trial 0 with value: inf.


...

Creating lag features for test set with lags: [7, 9]
Creating rolling stats for train set with windows: [13, 12, 15]
Creating rolling stats for val set with windows: [13, 12, 15]
Creating rolling stats for test set with windows: [13, 12, 15]
Using 20 features: ['TCW', 'TCLW', 'R250', 'R500', 'R850', 'U850', 'V850', 'EWSS', 'KX', 'CAPE', 'SSHF', 'PEV', 'AWS_lag7', 'AWS_lag9', 'AWS_rollmean_13', 'AWS_rollstd_13', 'AWS_rollmean_12', 'AWS_rollstd_12', 'AWS_rollmean_15', 'AWS_rollstd_15']
✅ Data shapes - Train: torch.Size([24595, 29, 20]), Val: torch.Size([5264, 29, 20]), Test: torch.Size([6823, 29, 20])
Error in fold fold_4: Target size (torch.Size([128, 2])) must be the same as input size (torch.Size([128, 1]))
Creating lag features for train set with lags: [7, 9]
Creating lag features for val set with lags: [7, 9]


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.28216725968774903 and num_layers=1
  warnings.warn(


Creating lag features for test set with lags: [7, 9]
Creating rolling stats for train set with windows: [13, 12, 15]
Creating rolling stats for val set with windows: [13, 12, 15]
Creating rolling stats for test set with windows: [13, 12, 15]
Using 20 features: ['TCW', 'TCLW', 'R250', 'R500', 'R850', 'U850', 'V850', 'EWSS', 'KX', 'CAPE', 'SSHF', 'PEV', 'AWS_lag7', 'AWS_lag9', 'AWS_rollmean_13', 'AWS_rollstd_13', 'AWS_rollmean_12', 'AWS_rollstd_12', 'AWS_rollmean_15', 'AWS_rollstd_15']


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.28216725968774903 and num_layers=1
  warnings.warn(
[I 2025-04-23 17:32:27,618] Trial 19 finished with value: inf and parameters: {'hidden_size': 42, 'num_layers': 1, 'dropout': 0.28216725968774903, 'lr': 0.001321033912105475, 'time_step_in': 29, 'time_step_out': 2, 'stride': 5, 'binary_weight': 1.5876341379100845, 'intensity_weight': 0.8579094716628972, 'regression_weight': 1.5396726126351952, 'rain_threshold': 0.19755933479288057, 'num_lags': 2, 'lag_0': 7, 'lag_1': 9, 'num_windows': 3, 'window_0': 13, 'window_1': 12, 'window_2': 15}. Best is trial 18 with value: 0.9371699094772339.


✅ Data shapes - Train: torch.Size([31161, 29, 20]), Val: torch.Size([5052, 29, 20]), Test: torch.Size([6638, 29, 20])
Error in fold fold_5: Target size (torch.Size([128, 2])) must be the same as input size (torch.Size([128, 1]))
✅ Best parameters for 2020-10:
  hidden_size: 33
  num_layers: 3
  dropout: 0.2603221931442648
  lr: 0.0014685413143977106
  time_step_in: 37
  time_step_out: 1
  stride: 5
  binary_weight: 1.9555325512038975
  intensity_weight: 0.7816173277290759
  regression_weight: 1.551441205543469
  rain_threshold: 0.07633697095473296
  num_lags: 1
  lag_0: 6
  num_windows: 2
  window_0: 3
  window_1: 6
💾 Best parameters saved to hybrid_best_params_2020-10.json

📦 Final evaluation on 2020-10, fold_1 test set

🚀 Training hybrid model for 2020-10, fold_1 with best parameters
Creating lag features for train set with lags: [6]
Creating lag features for val set with lags: [6]
Creating lag features for test set with lags: [6]
Creating rolling stats for train set with windows: [3